In [25]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from lifelines import CoxPHFitter


## Data Preprocessing    

In [26]:
# get data from s3
url = "https://s3.amazonaws.com/survival2024/hurricane.csv"
df = pd.read_csv(url)

In [27]:
#check for motor failure
df['motor']=0
df.loc[df['reason'] == 2, 'motor'] = 1

#grab hour columns but first rename hour column
df = df.rename(columns={'hour': 'failure_hour'})
hour_columns = [col for col in df.columns if col.startswith('h')]

#fill na's in the hour_columns with 0's, I'm assuming that since they are missing they are not turned on
df[hour_columns] = df[hour_columns].fillna(0)

#add pump numbers to df
df = df.reset_index(names='pump')

#melt df to reshape it
df_long = df.melt(
    id_vars=[col for col in df.columns if col not in hour_columns],  # Static columns to keep
    value_vars=hour_columns,  # Columns to unpivot
    var_name='run_hour_stop',  # New column name for hour labels
    value_name='running'  # Values from the hour columns
)

#grab numeric hour
df_long['run_hour_stop'] = df_long['run_hour_stop'].str.extract(r'(\d+)').astype(float).fillna(-1).astype(int)

#create a continous runtime column. how many hours has runnning=0 in a row
df_long['continuous_runtime'] = (
    df_long.groupby('pump')['running']
    .cumsum() * df_long['running']
)

df_long = df_long.sort_values(['pump', 'run_hour_stop']).reset_index(drop=True)
df_long.head()


,pump,backup,age,bridgecrane,servo,gear,trashrack,slope,elevation,survive,failure_hour,reason,reason2,motor,run_hour_stop,running,continuous_runtime
0,0,0,6.0,0,0,0,1,3,2,1,48,0,5,0,1,1.0,1.0
1,0,0,6.0,0,0,0,1,3,2,1,48,0,5,0,2,1.0,2.0
2,0,0,6.0,0,0,0,1,3,2,1,48,0,5,0,3,1.0,3.0
3,0,0,6.0,0,0,0,1,3,2,1,48,0,5,0,4,1.0,4.0
4,0,0,6.0,0,0,0,1,3,2,1,48,0,5,0,5,1.0,5.0


Now I'm going to grab only the columns in which I need to predict

In [28]:
df_motor = df_long[['pump', 'backup', 'age', 'bridgecrane', 'servo', 'gear', 'trashrack', 'slope', 'elevation', 'failure_hour', 'motor', 'run_hour_stop', 'running', 'continuous_runtime']]
df_motor = df_motor.copy()
df_motor['12+'] = (df_motor['continuous_runtime'] >= 12).astype(int)
df_motor['run_hour_start'] = df_motor['run_hour_stop']-1
df_motor.head()

,pump,backup,age,bridgecrane,servo,gear,trashrack,slope,elevation,failure_hour,motor,run_hour_stop,running,continuous_runtime,12+,run_hour_start
0,0,0,6.0,0,0,0,1,3,2,48,0,1,1.0,1.0,0,0
1,0,0,6.0,0,0,0,1,3,2,48,0,2,1.0,2.0,0,1
2,0,0,6.0,0,0,0,1,3,2,48,0,3,1.0,3.0,0,2
3,0,0,6.0,0,0,0,1,3,2,48,0,4,1.0,4.0,0,3
4,0,0,6.0,0,0,0,1,3,2,48,0,5,1.0,5.0,0,4


In [31]:
print(df_motor.columns)


Index(['pump', 'backup', 'age', 'bridgecrane', 'servo', 'gear', 'trashrack',
       'slope', 'elevation', 'failure_hour', 'motor', 'run_hour_stop',
       'running', 'continuous_runtime', '12+', 'run_hour_start'],
      dtype='object')


## Cox regression

In [36]:
cph = CoxPHFitter()
# cph.fit(df_motor.loc[:, ~df_motor.columns.isin(['run_hour', 'running', 'continuous_runtime'])], duration_col=['run_hour_start','run_hour_stop'], event_col='motor')
# cph.fit(df_motor,duration_col=['run_hour_start', 'run_hour_stop'], event_col='motor')
cph.fit(df_motor.loc[:, ~df_motor.columns.isin(['run_hour', 'running', 'continuous_runtime', 'run_hour_start', 'pump'])],duration_col='run_hour_stop', event_col='motor')
cph.print_summary()


c:\anaconda\envs\ds\Lib\site-packages\lifelines\utils\__init__.py:1120: ConvergenceWarning: Column trashrack have very low variance when conditioned on death event present or not. This may harm convergence. This could be a form of 'complete separation'. For example, try the following code:

>>> events = df['motor'].astype(bool)
>>> print(df.loc[events, 'trashrack'].var())
>>> print(df.loc[~events, 'trashrack'].var())

A very low variance means that the column trashrack completely determines whether a subject dies or not. See https://stats.stackexchange.com/questions/11109/how-to-deal-with-perfect-separation-in-logistic-regression.

  warnings.warn(dedent(warning_text), ConvergenceWarning)
c:\anaconda\envs\ds\Lib\site-packages\lifelines\fitters\coxph_fitter.py:1607: ConvergenceWarning: Newton-Raphson convergence completed successfully but norm(delta) is still high, 0.500. This may imply non-unique solutions to the maximum likelihood. Perhaps there is collinearity or complete separation 

<lifelines.CoxPHFitter: fitted with 36960 total observations, 31584 right-censored observations>
             duration col = 'run_hour_stop'
                event col = 'motor'
      baseline estimation = breslow
   number of observations = 36960
number of events observed = 5376
   partial log-likelihood = -45093.91
         time fit was run = 2024-11-20 03:31:26 UTC

---
               coef exp(coef)  se(coef)  coef lower 95%  coef upper 95% exp(coef) lower 95% exp(coef) upper 95%
covariate                                                                                                      
backup         0.02      1.02      0.03           -0.04            0.07                0.96                1.07
age           -0.81      0.45      0.02           -0.84           -0.77                0.43                0.46
bridgecrane   -0.11      0.89      0.04           -0.20           -0.03                0.82                0.97
servo          0.32      1.37      0.03            0.26            0.38                1.30                1.46
gear           0.27      1.32      0.05            0.18            0.37                1.19                1.45
trashrack    -19.00      0.00    196.09         -403.33          365.33                0.00           4.56e+158
slope         -0.31      0.73      0.01           -0.33           -0.29                0.72                0.75
elevation      0.09      1.10      0.02            0.06            0.13                1.06                1.14
failure_hour   0.01      1.01      0.00            0.01            0.02                1.01                1.02
12+           -0.32      0.73      0.03           -0.39           -0.25                0.68                0.78

              cmp to      z      p  -log2(p)
covariate                                   
backup          0.00   0.55   0.58      0.79
age             0.00 -42.78 <0.005       inf
bridgecrane     0.00  -2.73   0.01      7.32
servo           0.00  10.47 <0.005     82.85
gear            0.00   5.58 <0.005     25.30
trashrack       0.00  -0.10   0.92      0.12
slope           0.00 -33.49 <0.005    814.53
elevation       0.00   5.03 <0.005     20.95
failure_hour    0.00  10.57 <0.005     84.39
12+             0.00  -9.50 <0.005     68.68
---
Concordance = 0.89
Partial AIC = 90207.81
log-likelihood ratio test = 12711.81 on 10 df
-log2(p) of ll-ratio test = inf